# Setup

In [1]:
# Installs

%pip install -qU langchain-cohere langchain-community langchain-core==0.2.40 langchain_experimental langchain-google-community langchain-google-genai langchain-huggingface==0.0.3 langchain-openai langchain-qdrant langchain-google-vertexai
%pip install -qU -q chainlit==1.1.302
%pip install -qU -q docx2txt
%pip install -qU google-cloud-aiplatform
%pip install -qU google-cloud-discoveryengine
%pip install -qU IProgress
%pip install -qU nltk
%pip install -qU openpyxl
%pip install -qU pymupdf
%pip install -qU python-dotenv
%pip install -qU ragas==0.1.20
%pip install -qU tqdm
%pip install -qU unstructured 

# Verify installed packages have compatible dependencies
%pip check

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.28.2 requires protobuf<6.0,>=5.0, but you have protobuf 4.25.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.62.3 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.28.3 which is incompatible.
unstructured-client 0.28.0 requires aiofiles>=24.1.0, but you have aiofiles 23.2.1 which is incompatible.
google-ai-generativelanguage 0.6.6 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.28.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
N

In [2]:
# Download punkt_tab module that is used for sentence tokenizaiton

import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/dstampfli/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Environment Variables

In [4]:
# Get environment variables

import os
from dotenv import load_dotenv
import uuid

# Load environment variables from .env file
load_dotenv()

True

# Google Auth

In [6]:
# Set Google user permissions

import google.auth
import os
import sys

# If running in Colab, use the permissions of the currently authenticated user
if 'google.colab' in sys.modules:
	print('Running in Google Colab')
	from google.colab import auth

	auth.authenticate_user()

# If not, set the GOOGLE_APPLICATION_CREDENTIALS to the service account credentials file 
else:
	print('Running locally')
	# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'credentials.json'
	# print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

#####

def test_google_perms():
	from google.cloud import storage
	
	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])

	# Now, you can use the Google Cloud client libraries
	client = storage.Client(credentials=creds)

	# List all buckets in your project
	buckets = list(client.list_buckets())
	print(buckets)

test_google_perms()

Running locally
[<Bucket: 1033339822332_us_import_content_with_faq_csv>, <Bucket: mass-health-docs>, <Bucket: masshealth-docs>]


In [7]:
# Initialize Vertex AI

import vertexai

vertexai.init(project=os.environ['PROJECT_ID'], location=os.environ['REGION'])

In [8]:
# Verfify that our Google API key works

import requests

GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
url = f'https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={GOOGLE_API_KEY}'
  
headers = {
	"Content-Type": "application/json",
}
  
data = {"contents":[{"parts":[{"text":"What is MassHealth?"}]}]}

response = requests.post(url, headers=headers, json=data)
print(response.text)

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "MassHealth is the name of the Massachusetts state Medicaid program.  It provides health coverage to low-income adults, children, pregnant women, seniors, and people with disabilities.  Essentially, it's the state's version of Medicaid, offering a range of health care services to those who qualify based on income and other factors.\n"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "avgLogprobs": -0.13092216323403752
    }
  ],
  "usageMetadata": {
    "promptTokenCount": 5,
    "candidatesTokenCount": 68,
    "totalTokenCount": 73
  },
  "modelVersion": "gemini-1.5-flash-latest"
}



# Utils

In [10]:
# Load the docs from a directory

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/file_directory/

from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredExcelLoader

def process_directory(path: str, glob: str, loader_cls: str, use_multithreading=True):
	
	loader = DirectoryLoader(path=path, glob=glob, show_progress=True, loader_cls=loader_cls, use_multithreading=use_multithreading)
	
	docs = loader.load()
	
	return docs

#####

def test_process_directory():
	docs = []
	
	docs_pdf = process_directory('docs/masshealth/', '**/*.pdf', PyMuPDFLoader, True)
	# print(len(docs_pdf))
	docs_excel = process_directory('docs/masshealth/', '**/*.xlsx', UnstructuredExcelLoader, True)
	# print(len(docs_excel))

	docs.extend(docs_pdf)
	docs.extend(docs_excel)
	print(len(docs))

test_process_directory()

100%|██████████| 1/1 [00:00<00:00,  4.10it/s]
0it [00:00, ?it/s]

52


In [22]:
# Load the docs from a single file

# TODO - add TextLoader refernece
# TODO - add CSVLoader reference
# https://python.langchain.com/docs/integrations/document_loaders/pymupdf/
# https://python.langchain.com/docs/integrations/document_loaders/microsoft_word/

from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import Docx2txtLoader

def process_file(path: str):

	docs = None

	# Select the right loader
	if 'txt' in path.lower():
		loader = TextLoader(path)
	elif 'pdf' in path.lower():
		loader = PyMuPDFLoader(path)
	elif 'docx' in path.lower():
		loader = Docx2txtLoader(path)
	else:
		print(f'No document loader found for {path}')

	docs = loader.load()

	return docs

#####

def test_process_file():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	print(docs[0].metadata)

test_process_file()

{'source': 'docs/masshealth/ACA-1-0324.pdf', 'file_path': 'docs/masshealth/ACA-1-0324.pdf', 'page': 0, 'total_pages': 52, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Windows)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240228092333-05'00'", 'modDate': "D:20240229153600-05'00'", 'trapped': ''}


In [13]:
# Create embeddings using Hugging Face 

from langchain_huggingface import HuggingFaceEmbeddings

def create_embeddings_huggingface(model='Snowflake/snowflake-arctic-embed-m') -> HuggingFaceEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = HuggingFaceEmbeddings(model_name=model)

	return embeddings

#####

def test_create_embeddings_huggingface():
	text = 'What is my benefit for acupuncture?'
	embeddings = create_embeddings_huggingface()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_huggingface()

[-0.01862427406013012, 0.07942051440477371, -0.013963683508336544, -0.01928563043475151, 0.01954856514930725, 0.04386260360479355, 0.0017515963409096003, 0.0036999536678195, -0.03566894680261612, -0.039717864245176315, -0.01246454194188118, 0.02043858915567398, 0.017930103465914726, -0.004781828261911869, -0.03382826969027519, -0.010549322701990604, 0.06566154211759567, -0.018521374091506004, -0.024217912927269936, -0.02870921976864338, -0.020637696608901024, -0.01962466537952423, -0.004117612261325121, 0.036916013807058334, -0.023654824122786522, 0.03305623307824135, 0.03593701124191284, 0.05134713277220726, -0.07007366418838501, -0.0522121824324131, 0.029391717165708542, 0.03666974604129791, -0.01794552430510521, -0.06466533243656158, -0.01809859462082386, 0.007805256173014641, -0.03602747619152069, -0.03589760884642601, -0.0524902269244194, -0.00038496757042594254, -0.002180676208809018, 0.004602782893925905, 0.012346707284450531, 0.03780534490942955, -0.028559042140841484, -0.01321

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='Snowflake/snowflake-arctic-embed-m', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
# Create embeddings using OpenAI

from langchain_openai import OpenAIEmbeddings

def create_embeddings_openai(model="text-embedding-ada-002") -> OpenAIEmbeddings:

	# Initialize the OpenAIEmbeddings class
	embeddings = OpenAIEmbeddings(model=model)

	return embeddings

#####

def test_create_embeddings_openai():
	text = "What is my benefit for acupuncture?"
	embeddings = create_embeddings_openai()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_openai()

[-0.013142690062522888, -0.0037146771792322397, 0.043068721890449524, -0.04360708221793175, -0.03262455761432648, 0.013970416970551014, -0.02534325048327446, -0.019892366603016853, -0.016298819333314896, 0.003105658572167158, -0.0008849276346154511, 0.010551837272942066, 0.003590181702747941, 0.004108352120965719, -0.005821006838232279, -0.0023082143161445856, 0.05133253335952759, -0.009205939248204231, 0.005363401956856251, 0.005289377644658089, -0.013849286362528801, 0.0024579453747719526, 0.0037819722201675177, -0.0010405470384284854, -0.014024253003299236, 0.018075404688715935, 0.010585484094917774, -0.012173643335700035, -0.021776622161269188, -0.02169586904346943, 0.03650074079632759, 0.016419950872659683, -0.009576060809195042, -0.008479154668748379, -0.012173643335700035, 0.02253032475709915, 0.00017570272029843181, -0.009892347268760204, 0.022678373381495476, -0.0004090266884304583, 0.00913864467293024, 0.0032452954910695553, -0.0010472765425220132, 0.00490243174135685, -0.006

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x32bb8ea10>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x32bb9f6d0>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [16]:
# Create embeddings using Vertex AI

# https://python.langchain.com/docs/integrations/text_embedding/google_vertex_ai_palm/

from langchain_google_vertexai import VertexAIEmbeddings

def create_embeddings_vertexai(model="text-embedding-004") -> VertexAIEmbeddings:

	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])

	# Initialize the VertexAIEmbeddings class
	embeddings = VertexAIEmbeddings(model_name=model, 
								 credentials=creds)

	return embeddings

#####

def test_create_embeddings_vertexai():
	text = 'What is my benefit for acupuncture?'
	embeddings = create_embeddings_vertexai()
	vector = embeddings.embed_query(text)
	print(vector)
	return embeddings

test_create_embeddings_vertexai()

[0.037580445408821106, 0.033997565507888794, 0.006779782474040985, 0.013889092952013016, -0.01751645840704441, 0.05550519376993179, -0.007817636243999004, 0.012510783970355988, -0.03990676626563072, -0.0073811886832118034, -0.021110394969582558, 0.018350055441260338, 0.046891406178474426, 0.037558868527412415, -0.0948234349489212, -0.05031082406640053, -0.062025584280490875, -0.013999848626554012, -0.14326012134552002, -0.03631572052836418, 0.04239406809210777, 0.0033956223633140326, -0.012713606469333172, -0.023773323744535446, 0.0029369941912591457, 0.002878165803849697, -0.010965565219521523, 0.004967726767063141, 0.049845315515995026, -0.03375077620148659, -0.02053850144147873, 0.017944777384400368, 0.05141162872314453, -0.0443461574614048, 0.06225729361176491, 0.002635953249409795, 0.032474566251039505, 0.012381038628518581, 0.014667452313005924, -0.07240024954080582, -0.019168945029377937, -0.035043321549892426, -0.03396611660718918, 0.016433045268058777, -0.01277957670390606, -0

VertexAIEmbeddings(client=<vertexai.language_models.TextEmbeddingModel object at 0x32bb2a310>, async_client=None, project='davids-doghouse-398221', location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='text-embedding-004', full_model_name=None, client_options=ClientOptions: {'api_endpoint': 'us-central1-aiplatform.googleapis.com', 'client_cert_source': None, 'client_encrypted_cert_source': None, 'quota_project_id': None, 'credentials_file': None, 'scopes': None, 'api_key': None, 'api_audience': None, 'universe_domain': None}, api_endpoint=None, api_transport=None, default_metadata=(), additional_headers=None, client_cert_source=None, credentials=<google.oauth2.credentials.Credentials object at 0x308918c50>, client_preview=None, temperature=None, max_output_tokens=None, top_p=None, top_k=None, n=1, streaming=False, model_family=None, safety_settings=None, tuned_model_name=None, instance={'max_batch_size': 250, 'batch_size': 250, 'min_batch_size': 5, 'min_g

In [18]:
# Remove empty chunks 

from langchain.schema import Document

def remove_empty_chunks(chunks_start: list) -> list:
	
	start = len(chunks_start)
	# print(f'start - {start} chunks')
	
	# Remove empty chunks
	chunks_end = [chunk for chunk in chunks_start if chunk.page_content.strip()]

	end = len(chunks_end)
	# print(f'end - {end} chunks')

	return chunks_end   

#####

def test_remove_empty_chunks():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")

	# Added a test doc
	doc = Document(
		page_content='',
		metadata=docs[0].metadata
	)
	docs.append(doc)
	print(len(docs))

	# Remove the empty doc (chunk)
	docs = remove_empty_chunks(docs)
	print(len(docs))

test_remove_empty_chunks()

53
52


In [26]:
# Create a text splitter using recursive character text splitter

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/

from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_docs_recursive(documents: list, chunk_size=500, chunk_overlap=50) -> list:

	text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)

	chunks_start = text_splitter.split_documents(documents)

	chunks_end = remove_empty_chunks(chunks_start=chunks_start)

	return chunks_end

#####

def test_chunk_docs_recursive(): 
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_recursive(documents=docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_recursive()

385
MEMBER BOOKLET
for HEALTH and DENTAL COVERAGE 
and HELP PAYING COSTS
This is your member booklet for 
MassHealth, ConnectorCare Plans 
and Advance Premium Tax Credits, 
the Children’s Medical Security 
Plan, and the Health Safety Net. 
Commonwealth of Massachusetts
Executive Office of Health and Human Services
MassHealth | Massachusetts Health Connector
MARCH  2024


In [20]:
# Create a text splitter using NLTK

# https://python.langchain.com/docs/how_to/split_by_token/

from langchain.text_splitter import NLTKTextSplitter

def chunk_docs_nltk(documents: list, chunk_size=512, chunk_overlap=64) -> list:

	text_splitter = NLTKTextSplitter(
	chunk_size=chunk_size, 
	chunk_overlap=chunk_overlap)

	chunks_start = text_splitter.split_documents(documents)

	chunks_end = remove_empty_chunks(chunks_start=chunks_start)

	return chunks_end

#####

def test_chunk_docs_nltk(): 
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_nltk(documents=docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_nltk()


Created a chunk of size 993, which is longer than the specified 512
Created a chunk of size 740, which is longer than the specified 512
Created a chunk of size 553, which is longer than the specified 512
Created a chunk of size 585, which is longer than the specified 512
Created a chunk of size 782, which is longer than the specified 512
Created a chunk of size 545, which is longer than the specified 512
Created a chunk of size 646, which is longer than the specified 512
Created a chunk of size 576, which is longer than the specified 512
Created a chunk of size 551, which is longer than the specified 512
Created a chunk of size 529, which is longer than the specified 512
Created a chunk of size 610, which is longer than the specified 512
Created a chunk of size 877, which is longer than the specified 512
Created a chunk of size 610, which is longer than the specified 512
Created a chunk of size 566, which is longer than the specified 512
Created a chunk of size 575, which is longer tha

396
MEMBER BOOKLET
for HEALTH and DENTAL COVERAGE 
and HELP PAYING COSTS
This is your member booklet for 
MassHealth, ConnectorCare Plans 
and Advance Premium Tax Credits, 
the Children’s Medical Security 
Plan, and the Health Safety Net.

Commonwealth of Massachusetts
Executive Office of Health and Human Services
MassHealth | Massachusetts Health Connector
MARCH  2024


In [24]:
# Create a text splitter semantic chunking 

# https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/semantic-chunker/

from langchain_experimental.text_splitter import SemanticChunker

def chunk_docs_semantic(documents: list, ) -> list:

	# TODO - Use embeddings parameter
	text_splitter = SemanticChunker(create_embeddings_openai(), breakpoint_threshold_type="percentile")

	chunks_start = text_splitter.split_documents(documents)

	# Remove empty chunks
	chunks_end = remove_empty_chunks(chunks_start)

	return chunks_end

#####

def test_chunk_docs_semantic():
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_semantic(docs)
	print(len(chunks))
	print(chunks[0].page_content)

test_chunk_docs_semantic()

159
MEMBER BOOKLET
for HEALTH and DENTAL COVERAGE 
and HELP PAYING COSTS
This is your member booklet for 
MassHealth, ConnectorCare Plans 
and Advance Premium Tax Credits, 
the Children’s Medical Security 
Plan, and the Health Safety Net. Commonwealth of Massachusetts
Executive Office of Health and Human Services
MassHealth | Massachusetts Health Connector
MARCH  2024



In [ ]:
# Create a Qdrant vector store

from langchain_core.embeddings import Embeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

def create_qdrant_vector_store(location: str, collection_name: str, vector_size: int, embeddings: Embeddings, documents: list) -> QdrantVectorStore:

	# Initialize the Qdrant client
	qdrant_client = QdrantClient(location=location)

	# Create a collection in Qdrant
	qdrant_client.create_collection(collection_name=collection_name, vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE))

	# Initialize QdrantVectorStore with the Qdrant client
	qdrant_vector_store = QdrantVectorStore(client=qdrant_client, collection_name=collection_name, embedding=embeddings)
	
	qdrant_vector_store.add_documents(documents)
	
	return qdrant_vector_store

#####

def test_create_qdrant_vector_store():
	embeddings = create_embeddings_openai()
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	print(len(docs))
	chunks = chunk_docs_recursive(docs)
	print(len(chunks))
	vector_store = create_qdrant_vector_store(":memory:", "masshealth-test", 1536, embeddings, chunks)
	print(vector_store.collection_name)

test_create_qdrant_vector_store()

52
385
test


In [29]:
# Create a Qdrant retriever

# TODO - Add reference 

from langchain_core.retrievers import BaseRetriever
from langchain_qdrant import QdrantVectorStore

def create_retriever_qdrant(vector_store: QdrantVectorStore) -> BaseRetriever:

	retriever = vector_store.as_retriever()

	return retriever

#####

def test_create_retriever_qdrant(text: str = None):
	embeddings = create_embeddings_openai()
	docs = process_file("docs/masshealth/ACA-1-0324.pdf")
	chunks = chunk_docs_recursive(docs)
	vector_store = create_qdrant_vector_store(":memory:", "masshealth-test", 1536, embeddings, chunks)
	retriever = create_retriever_qdrant(vector_store)
	if text:
		docs = retriever.invoke(text)
		print(docs[0])

print('\nQDRANT')
test_create_retriever_qdrant('What is my benefit for acupuncture?')


QDRANT
page_content='11
Visit us online at www.MAhealthconnector.org
and enrolling in a Medicare prescription drug plan, 
see Section 10.)
	◆Acupuncture services**
	◆Behavioral health (mental health and addiction) 
services
	◆Well-child screenings (for children younger than 
21 years of age), including medical, vision, dental, 
hearing, behavioral health (mental health and 
addiction), and developmental screens, as well as 
shots
	◆Long-term-care services at home or in a long-term-' metadata={'source': 'docs/masshealth/ACA-1-0324.pdf', 'file_path': 'docs/masshealth/ACA-1-0324.pdf', 'page': 12, 'total_pages': 52, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Windows)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240228092333-05'00'", 'modDate': "D:20240229153600-05'00'", 'trapped': '', '_id': '99fb11a5c3a14104a3a9a59d5ac80073', '_collection_name': 'masshealth-test'}


In [30]:
# Create a Vertex AI retriever

# https://python.langchain.com/docs/integrations/retrievers/google_vertex_ai_search/

from langchain_google_community import VertexAISearchRetriever

def create_retriever_vertexai() -> VertexAISearchRetriever:

	retriever = VertexAISearchRetriever(project_id=os.environ['PROJECT_ID'], 
									 location_id=os.environ['LOCATION_ID'], 
									 data_store_id=os.environ['DATA_STORE_ID'], 
									 max_documents=3)

	return retriever

#####

def test_create_retriever_vertexai(text: str = None):
	retriever = create_retriever_vertexai()
	if text:
		docs = retriever.invoke(text)
		print(docs[0])

print('\nVERTEX AI')
test_create_retriever_vertexai('What is my benefit for acupuncture?')


VERTEX AI
page_content='Visit us online at www.MAhealthconnector.org 11

and enrolling in a Medicare prescription drug plan,
see Section 10.)
◆ Acupuncture services**
◆ Behavioral health (mental health and addiction)
services
◆ Well-child screenings ( for children younger than
21 years of age), including medical, vision, dental,
hearing, behavioral health (mental health and
addiction), and developmental screens, as well as
shots
◆ Long-term-care services at home or in a long-term
care facility, including home health services
◆ Transportation services***
◆ Quit-smoking services

*

Certain restrictions can be found in the MassHealth
regulations at 130 CMR 415.000.

**

Certain restrictions can be found in the MassHealth
regulations at 130 CMR 447.000.

***

Certain restrictions can be found in the MassHealth
regulations at 130 CMR 407.000.
Note: For disabled adults who also get Medicare Part B, MassHealth
will pay the Medicare premium, and if applicable, the
coinsurance and deductibles

In [ ]:
# Create a prompt template

# https://python.langchain.com/v0.1/docs/modules/model_io/prompts/quick_start/#chatprompttemplate
# https://python.langchain.com/v0.2/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html

from langchain.prompts import ChatPromptTemplate

def create_chat_prompt_template(template: str = None) -> ChatPromptTemplate:
	
	if template is None:
		template = '''
		You are a helpful conversational agent for the State of California.
		Your expertise is fully understanding the Medi-Cal provider manuals. 
		You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
		Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

		You will be analyzing the health plan documents to derive a good answer, based on the following information:
		1. The question asked.
		2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

		Now it's your turn!

		{question}

		{context}
		'''
	
	prompt = ChatPromptTemplate.from_template(template)

	return prompt

#####

def test_create_chat_prompt_template():
	prompt = create_chat_prompt_template()
	print(prompt)

test_create_chat_prompt_template()

input_variables=['context', 'question'] template="\n\tYou are a helpful conversational agent for the State of California.\n\tYour expertise is fully understanding the Medi-Cal provider manuals. \n\tYou need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  \n\tYour goal is to provide a helpful and detailed response, in at least 2-3 sentences. \n\n\tYou will be analyzing the health plan documents to derive a good answer, based on the following information:\n\t1. The question asked.\n\t2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.\n\n\tNow it's your turn!\n\n\t{question}\n\n\t{context}\n\t"


In [46]:
# Create a Langchain chain..

# https://python.langchain.com/docs/integrations/llms/google_ai/
# https://python.langchain.com/docs/integrations/chat/google_generative_ai/
# https://ai.google.dev/gemini-api/docs/safety-settings 

from google.generativeai.types import HarmCategory, HarmBlockThreshold
from langchain_core.output_parsers import StrOutputParser
from langchain_core.retrievers import BaseRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableSerializable
from langchain_google_genai import ChatGoogleGenerativeAI
from operator import itemgetter

def create_chain (model_name: str, 
				  prompt_template: ChatPromptTemplate, 
				  retriever: BaseRetriever) -> RunnableSerializable:

	if "gemini" in model_name.lower():
		llm = ChatGoogleGenerativeAI(
			model=model_name,
			temperature=0,
			safety_settings={
					HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
					HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
				},
			)
	else:
		print("Unsuported model name")
		
	chain = (
		{"context": itemgetter("question") | retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt_template | llm, "context": itemgetter("context")}
		)

	return chain

#####

def test_create_chain_qdrant():
	embeddings = create_embeddings_openai()
	docs = process_file('docs/medi-cal/acu.pdf')
	chunks = chunk_docs_recursive(docs)
	vector_store = create_qdrant_vector_store(":memory:", "test", 1536, embeddings, chunks)
	retriever = create_retriever_qdrant(vector_store)
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain('gemini-1.5-flash', chat_prompt_template, retriever)
	result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
	print(result)

print('\nQDRANT')
test_create_chain_qdrant()

###

def test_create_chain_vertexai():
	retreiver = create_retriever_vertexai()
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain('gemini-1.5-flash', chat_prompt_template, retreiver)
	result = chain.invoke({'question' : 'What is my benefit for acupuncture?'})
	print(result)

print('\nVERTEX AI')
test_create_chain_vertexai()


QDRANT
{'response': AIMessage(content='Medi-Cal covers acupuncture services for the treatment of severe, persistent chronic pain stemming from a generally recognized medical condition.  However,  coverage is limited to treatment by an enrolled physician, dentist, podiatrist, or certified acupuncturist, and is subject to a limit of two services per calendar month, or a combination of two services from a list of specified therapies including acupuncture.  For more details, including billing information and non-covered services, please refer to the complete Medi-Cal Acupuncture Services manual.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1ffb94b6-e672-4751-b099-5221afa8b52d-0', usage_metadata={'input_tokens': 1513, 'output_tokens': 105, 'total_tokens': 1618}), 'context': [Document(metadata={'source': 'docs/medi-cal/acu.pdf', 'file_path': 'docs/medi-cal/acu.pdf', 'page': 0, 'total_pages': 4, '

I0000 00:00:1732714120.067952  724137 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


{'response': AIMessage(content='Based on the provided document,  the MassHealth plan covers acupuncture services. However, the document notes that "Certain restrictions can be found in the MassHealth regulations at 130 CMR 447.000."  To understand your specific benefits and any limitations, you will need to review the MassHealth regulations cited or contact MassHealth directly for clarification.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d6dbfa93-c32b-47d2-affc-5ff9d888516b-0', usage_metadata={'input_tokens': 1312, 'output_tokens': 76, 'total_tokens': 1388}), 'context': [Document(metadata={'id': '6906324640445825d0cf4505b0dcef86', 'source': 'gs://masshealth-docs/ACA-1-0324.pdf:13'}, page_content='Visit us online at www.MAhealthconnector.org 11\n\nand enrolling in a Medicare prescription drug plan,\nsee Section 10.)\n◆ Acupuncture services**\n◆ Behavioral health (mental health and addiction

In [57]:
# Generate answers from a chain usin a list of questions

import json
from langchain_core.runnables import RunnableSerializable
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser


def generate_answers_contexts(chain: RunnableSerializable, 
							  questions: list):
	
	answers = []
	contexts = []

	# Loop over the list of questions and call the chain to get the answer and context
	for question in questions:
		print(question)

		# Call the chain to get answers and contexts
		response = chain.invoke({"question" : question})
		print(response)

		# Capture the answer and context 
		answers.append(response["response"].content)
		contexts.append([context.page_content for context in response["context"]])

	return answers, contexts

#####

def test_generate_answers_contexts():
	retreiver = create_retriever_vertexai()
	chat_prompt_template = create_chat_prompt_template()
	chain = create_chain(model_name="gemini-1.5-flash", 
					  prompt_template=chat_prompt_template,
					  retriever=retreiver)
	
	questions = ["What is my benefit for acupuncture?",
			  "Who should I call if I have an emergency?",
			  "Provide an overview of the newborn hearing screening program?"]
	
	answers, contexts = generate_answers_contexts(chain=chain, questions=questions)
	
	print(f"Total number of answers = {len(answers)}")
	print(f"Total number of contexts = {len(contexts)}")

test_generate_answers_contexts()

I0000 00:00:1732714935.554291  724137 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


What is my benefit for acupuncture?
{'response': AIMessage(content='Based on the provided document,  the document does list acupuncture as a covered service under MassHealth. However,  it explicitly states that "Certain restrictions can be found in the MassHealth regulations at 130 CMR 447.000."  To understand your specific benefits and any limitations, you must refer to the MassHealth regulations cited (130 CMR 447.000) for complete details on acupuncture coverage.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-073138e6-2731-4cea-88be-f359d35a47a7-0', usage_metadata={'input_tokens': 1312, 'output_tokens': 94, 'total_tokens': 1406}), 'context': [Document(metadata={'id': '6906324640445825d0cf4505b0dcef86', 'source': 'gs://masshealth-docs/ACA-1-0324.pdf:13'}, page_content='Visit us online at www.MAhealthconnector.org 11\n\nand enrolling in a Medicare prescription drug plan,\nsee Section 10.)\n◆ 

In [58]:
# Run a Ragas evaluation 

from datasets import Dataset
from langchain_core.runnables import RunnableSerializable
from pandas import DataFrame
from ragas import evaluate
from ragas.metrics import (faithfulness, answer_relevancy, answer_correctness, context_recall, context_precision)

def run_ragas_evaluation(chain: RunnableSerializable, 
						 questions: list, 
						 groundtruths: list, 
						 eval_metrics: list = [answer_correctness, answer_relevancy, context_recall, context_precision, faithfulness]):
	
	answers = []
	contexts = []
	answers, contexts = generate_answers_contexts(chain=chain, 
											   questions=questions)

	# Create the input dataset 
	input_dataset = Dataset.from_dict({
	"question" : questions,         # From the dataframe
	"answer" : answers,             # From the chain
	"contexts" : contexts,          # From the chain
	"ground_truth" : groundtruths   # From the dataframe
	})

	# Run the Ragas evaluation using the input dataset and eval metrics
	ragas_results = evaluate(input_dataset, eval_metrics)
	ragas_results_df = ragas_results.to_pandas()
	
	return ragas_results, ragas_results_df
	
#####

def test_run_ragas_evaluation():
	print("test")    

test_run_ragas_evaluation()

test


In [59]:
# Query Vertex AI Search datastore using HTTP Post

import json
import google.auth
from google.auth.transport.requests import Request
import requests

def query_chunks(query: str, 
				 n: int=5):
		
	PROJECT_ID = os.environ['PROJECT_ID']
	LOCATION_ID = os.environ['LOCATION_ID']
	DATA_STORE_ID = os.environ['DATA_STORE_ID']

	if LOCATION_ID == 'us':
		api_endpoint = 'us-discoveryengine.googleapis.com'
	else:
		api_endpoint = 'discoveryengine.googleapis.com'

	url = f"https://{api_endpoint}/v1alpha/projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search:search"
	print(url)
	
	headers = {
			"Authorization": f"Bearer {access_token}",
			"Content-Type": "application/json",
	}
	
	post_data = {
			"servingConfig": f"projects/{PROJECT_ID}/locations/{LOCATION_ID}/collections/default_collection/dataStores/{DATA_STORE_ID}/servingConfigs/default_search",
			"pageSize": n,
			"query": query,
			"contentSearchSpec": {"searchResultMode": "CHUNKS"},
	}
	
	response = requests.post(url, headers=headers, json=post_data)

	if response.status_code != 200:
		print(
				f"Error retrieving search results: {response.status_code} -"
				f" {response.text}"
		)

	return response.json()

#####

def test_query_chunks():
	creds, _ = google.auth.default()
	creds.refresh(Request())
	access_token = creds.token
	# print(access_token)
	response = query_chunks('What is my benefit for acupuncture?')
	print(json.dumps(response, indent=4))

test_query_chunks()

I0000 00:00:1732714969.690561  724137 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


https://us-discoveryengine.googleapis.com/v1alpha/projects/davids-doghouse-398221/locations/us/collections/default_collection/dataStores/masshealth_1732658612644/servingConfigs/default_search:search
{
    "results": [
        {
            "chunk": {
                "name": "projects/1033339822332/locations/us/collections/default_collection/dataStores/masshealth_1732658612644/branches/0/documents/6906324640445825d0cf4505b0dcef86/chunks/13",
                "id": "13",
                "content": "Visit us online at www.MAhealthconnector.org 11\n\nand enrolling in a Medicare prescription drug plan,\nsee Section 10.)\n\u25c6 Acupuncture services**\n\u25c6 Behavioral health (mental health and addiction)\nservices\n\u25c6 Well-child screenings ( for children younger than\n21 years of age), including medical, vision, dental,\nhearing, behavioral health (mental health and\naddiction), and developmental screens, as well as\nshots\n\u25c6 Long-term-care services at home or in a long-term\ncare 

# Create RAG chain using Google Vertex AI

In [61]:
# Build RAG chain using Vertex AI Agent Builder datastore

retreiver = create_retriever_vertexai()
chat_prompt_template = create_chat_prompt_template()
chain = create_chain(model_name='gemini-1.5-flash', 
                     prompt_template=chat_prompt_template, 
                     retriever=retreiver)

I0000 00:00:1732714996.781757  724137 fork_posix.cc:75] Other threads are currently calling into gRPC, skipping fork() handlers


In [62]:
# Test the chain 

questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'Provide an overview of the newborn hearing screening program?',]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)

What is my benefit for acupuncture?
{'response': AIMessage(content='Based on the provided document,  the MassHealth plan covers acupuncture services. However, the document notes that "Certain restrictions can be found in the MassHealth regulations at 130 CMR 447.000."  To understand your specific benefits and any limitations, you will need to review the MassHealth regulations cited or contact MassHealth directly for clarification.\n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-e22d2c50-50f1-4610-a806-a836d7810777-0', usage_metadata={'input_tokens': 1312, 'output_tokens': 76, 'total_tokens': 1388}), 'context': [Document(metadata={'id': '6906324640445825d0cf4505b0dcef86', 'source': 'gs://masshealth-docs/ACA-1-0324.pdf:13'}, page_content='Visit us online at www.MAhealthconnector.org 11\n\nand enrolling in a Medicare prescription drug plan,\nsee Section 10.)\n◆ Acupuncture services**\n◆ Behavioral

# Create synthetic testset

In [ ]:
# https://docs.ragas.io/en/latest/references/generate/
# https://docs.ragas.io/en/stable/howtos/customizations/_run_config/

import google.auth
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings
from ragas.testset.evolutions import simple, reasoning, multi_context
from ragas.testset.generator import RunConfig
from ragas.testset.generator import TestsetGenerator

testset_name = "masshealth_testset.csv"
use_chunking = False
use_vertexai = False

# Load the docs
docs = []
docs_pdf = process_directory('docs/masshealth/', '**/*.pdf', PyMuPDFLoader, True)
docs.extend(docs_pdf)
print(len(docs))

if use_chunking:
	# Chunk the docs 
	docs = chunk_docs_recursive(docs)
	# docs = chunk_docs_nltk(docs)
	# docs = chunk_docs_semantic(docs)

# Set up the parameters for generating the testset
if use_vertexai:
	testset_name = "masshealth_vertexai_testset.csv"

	# https://docs.ragas.io/en/v0.1.21/howtos/customisations/gcp-vertexai.html
	creds, _ = google.auth.default(quota_project_id=os.environ["PROJECT_ID"])
	generator_llm = ChatVertexAI(credentials=creds,
							  model_name="gemini-1.5-pro")
	critic_llm = ChatVertexAI(credentials=creds,
							model_name="gemini-1.0-pro")
	embeddings = VertexAIEmbeddings(credentials=creds, 
								 model_name="textembedding-gecko")
else:
	testset_name = "masshealth_openai_testset.csv"

	generator_llm = ChatOpenAI(model="gpt-4o")
	critic_llm = ChatOpenAI(model="gpt-3.5-turbo-16k")
	embeddings = OpenAIEmbeddings()

run_config=RunConfig(
	max_workers=2	# default: 16 but smaller number is required to avoid rate limits
	)

generator = TestsetGenerator.from_langchain(generator_llm=generator_llm, 
											critic_llm=critic_llm, 
											embeddings=embeddings,
											run_config=run_config)

test_size=50

distributions = {simple: 0.5, 
				 multi_context: 0.4, 
				 reasoning: 0.1}

# Generate the testset and save to disk 
testset = generator.generate_with_langchain_docs(documents=docs, 
												 test_size=25, 
												 distributions=distributions)

# Write the testet to disk 
print(f"Writing {testset_name}")
testset_df = testset.to_pandas()
testset_df.to_csv(f"testsets/{testset_name}")
testset_df

# Evaluate synthetic dataset using Ragas

In [ ]:
# Evaluate the synthetic testset using Ragas

import pandas as pd

# Run the Ragas evaluation and show the results
# Get the questions and groundtruths from the dataframe
testset_df = pd.read_csv("testsets/synthetic_testset.csv")

questions = testset_df["question"].values.tolist()
questions = [str(question) for question in questions]

groundtruths = testset_df["ground_truth"].values.tolist()
groundtruths = [str(ground_truth) for ground_truth in groundtruths]  

# Specify the eval metrics
eval_metrics = [answer_correctness, answer_relevancy, context_precision, context_recall, faithfulness]

# Run the Ragas evaluation and show the results
ragas_results, ragas_results_df = run_ragas_evaluation(chain, questions, groundtruths, eval_metrics)

# Write the results to disk
print("Writing synthetic_testset_ragas_results.csv")
ragas_results_df.to_csv("ragas/synthetic_testset_ragas_results.csv")

# Show the resutls
ragas_results

# Create sample ReAct agent using Gemini

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools
from langchain_google_genai import ChatGoogleGenerativeAI
from google.generativeai.types import HarmCategory, HarmBlockThreshold

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
							 temperature=0, 
							 safety_settings={
								 HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
								 HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH
								 }
								 )

tools = load_tools(["arxiv"])

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "What's the paper 1605.08386 about?"})

# Use Unstructured to parse docs and load to Faiss

In [ ]:
# https://colab.research.google.com/drive/1BJYYyrPVe0_9EGyXqeNyzmVZDrCRZwsg?usp=sharing#scrollTo=CwzrH-9_K6-z

%pip install -qU "unstructured-ingest[pdf]" unstructured faiss-cpu

In [ ]:
from unstructured_ingest.v2.pipeline.pipeline import Pipeline
from unstructured_ingest.v2.interfaces import ProcessorConfig
from unstructured_ingest.v2.processes.connectors.local import (
	LocalIndexerConfig,
	LocalDownloaderConfig,
	LocalConnectionConfig,
	LocalUploaderConfig
)
from unstructured_ingest.v2.processes.partitioner import PartitionerConfig
from unstructured_ingest.v2.processes.chunker import ChunkerConfig

In [ ]:
import os 

directory_with_pdfs="app_2/content/data"
directory_with_results="app_2/content/output"

In [ ]:
Pipeline.from_configs(
	context=ProcessorConfig(),
	indexer_config=LocalIndexerConfig(input_path=directory_with_pdfs),
	downloader_config=LocalDownloaderConfig(),
	source_connection_config=LocalConnectionConfig(),
	partitioner_config=PartitionerConfig(
		partition_by_api=True,
		api_key=os.getenv("UNSTRUCTURED_API_KEY"),
		partition_endpoint=os.getenv("UNSTRUCTURED_API_URL"),
		strategy="hi_res",
		additional_partition_args={
			"split_pdf_page": True,
			"split_pdf_concurrency_level": 15,
			},
		),
	uploader_config=LocalUploaderConfig(output_dir=directory_with_results)
).run()

In [ ]:
from unstructured.staging.base import elements_from_json

elements = []
for filename in os.listdir(directory_with_results):
	if filename.endswith('.json'):
		file_path = os.path.join(directory_with_results, filename)
		try:
			elements.extend(elements_from_json(filename=file_path))
		except IOError:
			print(f"Error: Could not read file {filename}.")
print(len(elements))

In [ ]:
from langchain.schema import Document

docs = []

for element in elements:
	metadata = element.metadata.to_dict()
	docs.append(Document(page_content=element.text, metadata=metadata))

print(len(docs))

In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
vector_store.save_local("app_2/faiss_index_unstructured")

In [ ]:
new_vector_store = FAISS.load_local("app_2/faiss_index_unstructured", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
new_chain = create_chain('gemini-1.5-flash', chat_prompt_template, new_retriever)

In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
	print(question)
	result = chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)

# Using PyMuPDFLoader to parse docs and load to Faiss

In [ ]:
docs = []

docs_pdf = process_directory('docs/medi-cal/', '**/*.pdf', PyMuPDFLoader, True)
docs_excel = process_directory('docs/medi-cal/', '**/*.xlsx', UnstructuredExcelLoader, True)

docs.extend(docs_pdf)
docs.extend(docs_excel)

print(len(docs))



In [ ]:
from langchain.vectorstores import FAISS

vector_store = FAISS.from_documents(documents=docs, embedding=create_embeddings_vertexai())
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
vector_store.save_local("app_2/faiss_index_pymupdfloader")

In [ ]:
new_vector_store = FAISS.load_local("app_2/faiss_index_pymupdfloader", create_embeddings_vertexai(), allow_dangerous_deserialization=True)
new_retriever = new_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [ ]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank

compressor = CohereRerank(model="rerank-english-v3.0")
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=new_retriever)

In [ ]:
template = '''
You are a helpful conversational agent for the State of California.
Your expertise is fully understanding the Medi-Cal provider manuals. 
You need to answer questions posed by a member, who is trying to get answers about services provided by Medi-Cal.  
Your goal is to provide a helpful and detailed response, in at least 2-3 sentences. 

You will be analyzing the health plan documents to derive a good answer, based on the following information:
1. The question asked.
2. The provided context, which comes from health plan document. You will need to answer the question based on the provided context and the conversation history.

Now it's your turn!

{question}

{context}
'''

prompt = PromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(
	model='gemini-1.5-flash',
	temperature=0,
	safety_settings={
			HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
			HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_ONLY_HIGH,
		}
	)
		
# new_chain = (
#         {"context": itemgetter("question") | new_retriever, "question": itemgetter("question")} 
#         | RunnablePassthrough.assign(context=itemgetter("context")) 
#         | {"response": prompt | llm, "context": itemgetter("context")}
#         )

new_chain = (
		{"context": itemgetter("question") | compression_retriever, "question": itemgetter("question")} 
		| RunnablePassthrough.assign(context=itemgetter("context")) 
		| {"response": prompt | llm, "context": itemgetter("context")}
		)


In [ ]:
questions = ['What is my benefit for acupuncture?',
'Who should I call if I have an emergency?',
'What is my vision benefit?',]

for question in questions:
	print(question)
	result = new_chain.invoke({"question" : question})
	print(result)
	print(result["response"].content)